- Official docs: https://pandas.pydata.org/docs/development/extending.html
- StackOverflow: https://stackoverflow.com/a/68972163/247482
- Arrow integration: https://arrow.apache.org/docs/python/extending_types.html#defining-extension-types-user-defined-types


In [1]:
from uuid import uuid4

import pandas as pd

from pandas_uuid import UuidDtype, UuidExtensionArray

In [ ]:
UuidExtensionArray(values=[0, uuid4()])

<UuidExtensionArray>
[UUID('00000000-0000-0000-0000-000000000000'), UUID('a51d5ad6-efdc-43bc-970f-c46ad2697b12')]
Length: 2, dtype: uuid

In [3]:
s = pd.Series([uuid4()], dtype=UuidDtype(), name="s")
s

0    90e22661-0afe-4490-9103-b244fbf3e60e
Name: s, dtype: uuid

In [4]:
pd.DataFrame(s)

,s
0,90e22661-0afe-4490-9103-b244fbf3e60e


pyarrow integration

In [ ]:
import pyarrow as pa

arr = pa.Array.from_pandas(s.array)
arr

ModuleNotFoundError: No module named 'pyarrow'

In [ ]:
# TODO: can pandas be convinced to use this dtype automatically?
pd.Series(arr, dtype=UuidDtype())

0    6f913769-35bf-4578-a9e6-ffecaa381cfd
dtype: uuid